<a href="https://colab.research.google.com/github/Sugina99/Deep_Learning_Studies/blob/main/CNN_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt


In [4]:
_URL='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip=tf.keras.utils.get_file('/content/drive/MyDrive/LibrariesForAI/Deep_learning/CNN/cats_and_dogs.zip',origin=_URL,extract=True)

In [6]:
PATH=os.path.join(os.path.dirname(path_to_zip),'cats_and_dogs.zip')
PATH

'/content/drive/MyDrive/LibrariesForAI/Deep_learning/CNN/cats_and_dogs.zip'

In [8]:
# PATH=os.path.join(os.path.dirname(path_to_zip),'cats_and_dogs.zip')
# New_PATH='/content/drive/MyDrive/LibrariesForAI/Deep_learning/CNN/cats_and_dogs'

# !unzip $PATH -d $New_PATH

In [9]:
DataPath='/content/drive/MyDrive/LibrariesForAI/Deep_learning/CNN/cats_and_dogs/cats_and_dogs_filtered'
DataPath

'/content/drive/MyDrive/LibrariesForAI/Deep_learning/CNN/cats_and_dogs/cats_and_dogs_filtered'

In [13]:
! ls $DataPath

train  validation  vectorize.py


In [14]:
train_dir=os.path.join(DataPath,'train')
validation_dir=os.path.join(DataPath,'validation')

In [17]:
TrainCat_dir=os.path.join(train_dir,'cats') #directory of training cat pics
TrainDog_dir=os.path.join(train_dir,'dogs') #directory of training dog pics
ValidationCat_dir=os.path.join(validation_dir,'cats') #directory of validation cat pics
ValidationDog_dir=os.path.join(validation_dir,'dogs') #directory of validation dogs pics

In [19]:
num_cat_tr=len(os.listdir(TrainCat_dir))
num_dog_tr=len(os.listdir(TrainDog_dir))
num_cat_val=len(os.listdir(ValidationCat_dir))
num_dog_val=len(os.listdir(ValidationDog_dir))

total_train=num_cat_tr + num_dog_tr
total_validation=num_cat_val + num_dog_val

In [21]:
print("total training cat images : ",num_cat_tr)
print("total training dog images : ",num_dog_tr)
print("------------------------------------")
print("total validation cat images : ",num_cat_val)
print("total validation dog images : ",num_dog_val)
print("------------------------------------")
print("total training images : ",total_train)
print("total validation images : ",total_validation)

total training cat images :  1000
total training dog images :  1000
------------------------------------
total validation cat images :  500
total validation dog images :  500
------------------------------------
total training images :  2000
total validation images :  1000


In [23]:
batch_size=128
epochs=30
IMG_HEIGHT=150
IMG_WIDTH=150

In [24]:
train_image_generator=ImageDataGenerator(rescale=1./255) # generator for training data
validation_image_generator=ImageDataGenerator(rescale=1./255) # generator for validation data

In [26]:
train_image_generator.flow_from_directory(batch_size=batch_size,
                                          directory=train_dir,
                                          shuffle=True,
                                          target_size=(IMG_HEIGHT,IMG_WIDTH),
                                          class_mode='binary') # multiclass---> class_mode='categorical'

Found 2000 images belonging to 2 classes.
